<a href="https://colab.research.google.com/github/NeegarNaushabaIqbal/ClimateColaboratory/blob/main/BiharTemp%26RainMSR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages("ncdf4")
install.packages("tidyverse")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(ncdf4)
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [ ]:
nc_data_Bihar <- nc_open("Bihar.nc")

In [ ]:
print(nc_data_Bihar)

File Bihar.nc (NC_FORMAT_NETCDF4):

     5 variables (excluding dimension variables):
        8 byte int number[]   (Contiguous storage)  
            long_name: ensemble member numerical id
            units: 1
            standard_name: realization
        string expver[valid_time]   (Contiguous storage)  
        float t[longitude,latitude,pressure_level,valid_time]   (Chunking: [20,13,1,420])  (Compression: shuffle,level 1)
[1] ">>>> WARNING <<<  attribute GRIB_paramId is an 8-byte value, but R"
[1] "does not support this data type. I am returning a double precision"
[1] "floating point, but you must be aware that this could lose precision!"
[1] ">>>> WARNING <<<  attribute GRIB_numberOfPoints is an 8-byte value, but R"
[1] "does not support this data type. I am returning a double precision"
[1] "floating point, but you must be aware that this could lose precision!"
[1] ">>>> WARNING <<<  attribute GRIB_stepUnits is an 8-byte value, but R"
[1] "does not support this data type. I am

In [ ]:
longitude <- ncvar_get(nc_data_Bihar, "longitude")
latitude <- ncvar_get(nc_data_Bihar, "latitude")
pressure_level <- ncvar_get(nc_data_Bihar, "pressure_level")
time <- ncvar_get(nc_data_Bihar, "valid_time")

# Convert time units
time_units <- ncatt_get(nc_data_Bihar, "valid_time", "units")$value
time <- as.POSIXct(time, origin = "1970-01-01", tz = "UTC")

# Load data variables
temperature <- ncvar_get(nc_data_Bihar, "t") # 4D array: [lon, lat, pressure, time]
rel_humidity <- ncvar_get(nc_data_Bihar, "r") # relative humidity [%]
spec_humidity <- ncvar_get(nc_data_Bihar, "q") # specific humidity [kg/kg]

# Reshape dimensions into a data frame
df <- expand.grid(
  longitude = longitude,
  latitude = latitude,
  pressure_level = pressure_level,
  time = time
)

# Flatten 4D arrays and add as columns
df$temperature <- as.vector(temperature)
df$relative_humidity <- as.vector(rel_humidity)
df$specific_humidity <- as.vector(spec_humidity)

# Inspect result
head(df)

,longitude,latitude,pressure_level,time,temperature,relative_humidity,specific_humidity
,<dbl[1d]>,<dbl[1d]>,<dbl[1d]>,<dttm[1d]>,<dbl>,<dbl>,<dbl>
1,83.32400,27.287,1000,1990-01-01,291.2277,68.56175,0.008508403
2,83.57405,27.287,1000,1990-01-01,291.3602,70.31810,0.008614916
3,83.82411,27.287,1000,1990-01-01,291.7045,71.80101,0.008702595
4,84.07416,27.287,1000,1990-01-01,292.1383,71.15160,0.008698065
5,84.32421,27.287,1000,1990-01-01,292.5212,70.44749,0.008646268
6,84.57426,27.287,1000,1990-01-01,292.6642,69.84251,0.008642871


In [ ]:
install.packages("lubridate")
library(lubridate)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [ ]:
install.packages("zoo")  # Run if not installed
library(zoo)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric




In [ ]:
df <- df %>%
  mutate(
    time = as.yearmon(time, "%Y-%m-%d"),  # Converts to "Jan 1990" format
    temperature_c = temperature - 273.15  # Convert to Celsius
  )

# View the modified dataframe
head(df)

,longitude,latitude,pressure_level,time,temperature,relative_humidity,specific_humidity,temperature_c
,<dbl[1d]>,<dbl[1d]>,<dbl[1d]>,<yearmon>,<dbl>,<dbl>,<dbl>,<dbl>
1,83.32400,27.287,1000,Jan 1990,291.2277,68.56175,0.008508403,18.07766
2,83.57405,27.287,1000,Jan 1990,291.3602,70.31810,0.008614916,18.21023
3,83.82411,27.287,1000,Jan 1990,291.7045,71.80101,0.008702595,18.55447
4,84.07416,27.287,1000,Jan 1990,292.1383,71.15160,0.008698065,18.98831
5,84.32421,27.287,1000,Jan 1990,292.5212,70.44749,0.008646268,19.37124
6,84.57426,27.287,1000,Jan 1990,292.6642,69.84251,0.008642871,19.51425


In [ ]:
df_spatial_avg <- df %>%
  group_by(time) %>%
  summarise(
    avg_temp_c = mean(temperature_c, na.rm = TRUE),
    sd_temp_c = sd(temperature_c, na.rm = TRUE),
    cv_temp_c = sd_temp_c / avg_temp_c,

    avg_rel_humidity = mean(relative_humidity, na.rm = TRUE),
    sd_rel_humidity = sd(relative_humidity, na.rm = TRUE),
    cv_rel_humidity = sd_rel_humidity / avg_rel_humidity,

    avg_spec_humidity = mean(specific_humidity, na.rm = TRUE),
    sd_spec_humidity = sd(specific_humidity, na.rm = TRUE),
    cv_spec_humidity = sd_spec_humidity / avg_spec_humidity,
    .groups = "drop"
  ) %>%
  mutate(
    year = year(time),  # Extracts the year
    month = month(time, label = TRUE, abbr = TRUE),  # Extracts the month
    yearmonth = as.yearmon(time)  # Combines year and month
  )
df_spatial_avg

time,avg_temp_c,sd_temp_c,cv_temp_c,avg_rel_humidity,sd_rel_humidity,cv_rel_humidity,avg_spec_humidity,sd_spec_humidity,cv_spec_humidity,year,month,yearmonth
<yearmon>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<ord>,<yearmon>
Jan 1990,19.33634,0.7966955,0.04120199,64.17878,5.541296,0.08634156,0.008153793,0.0003931764,0.04822006,1990,Jan,Jan 1990
Feb 1990,20.27928,0.8034896,0.03962120,63.96904,4.067586,0.06358679,0.008622389,0.0005780407,0.06703951,1990,Feb,Feb 1990
Mar 1990,24.04149,0.8983804,0.03736792,53.87720,6.599863,0.12249827,0.008888774,0.0009458913,0.10641415,1990,Mar,Mar 1990
Apr 1990,29.79853,2.0819888,0.06986885,45.24342,11.431962,0.25267680,0.009759053,0.0014753555,0.15117815,1990,Apr,Apr 1990
May 1990,29.63234,1.6729635,0.05645736,70.41598,7.978035,0.11329864,0.016878349,0.0013107042,0.07765594,1990,May,May 1990
Jun 1990,30.38284,1.4570396,0.04795600,76.09963,8.002538,0.10515870,0.019070562,0.0011897559,0.06238704,1990,Jun,Jun 1990
Jul 1990,27.99853,0.4612983,0.01647580,88.04368,1.351272,0.01534775,0.019594474,0.0010068715,0.05138548,1990,Jul,Jul 1990
Aug 1990,28.64218,0.3796141,0.01325368,84.37242,1.798621,0.02131764,0.019544322,0.0010943722,0.05599438,1990,Aug,Aug 1990
Sep 1990,27.53694,0.4080383,0.01481785,85.91865,1.257775,0.01463914,0.018716469,0.0011422963,0.06103161,1990,Sep,Sep 1990


In [ ]:
maxTempCV <- max(df_spatial_avg$cv_temp_c, na.rm = TRUE)
minTempCV <- min(df_spatial_avg$cv_temp_c, na.rm = TRUE)
maxSpecHumCV <- max(df_spatial_avg$cv_spec_humidity, na.rm = TRUE)
minSpecHumCV <- min(df_spatial_avg$cv_spec_humidity, na.rm = TRUE)
maxRelHumCV <- max(df_spatial_avg$cv_rel_humidity, na.rm = TRUE)
minRelHumCV <- min(df_spatial_avg$cv_rel_humidity, na.rm = TRUE)

In [ ]:
maxTempCV
minTempCV

[1] 0.09133817

[1] 0.00948231

In [ ]:
maxSpecHumCV
minSpecHumCV

[1] 0.2294181

[1] 0.03758538

In [ ]:
maxRelHumCV
minRelHumCV

[1] 0.3167141

[1] 0.01255337

In [ ]:
unique(df_spatial_avg$year)

[1] 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004
[16] 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019
[31] 2020 2021 2022 2023 2024

In [ ]:
length(unique(df_spatial_avg$year))

[1] 35

In [ ]:
nc_data_BiharRain <- nc_open("BiharRain1.nc")

In [ ]:
print(nc_data_BiharRain)

File BiharRain1.nc (NC_FORMAT_NETCDF4):

     3 variables (excluding dimension variables):
        8 byte int number[]   (Contiguous storage)  
            long_name: ensemble member numerical id
            units: 1
            standard_name: realization
        string expver[valid_time]   (Contiguous storage)  
        float tp[longitude,latitude,valid_time]   (Chunking: [20,13,420])  (Compression: shuffle,level 1)
[1] ">>>> WARNING <<<  attribute GRIB_paramId is an 8-byte value, but R"
[1] "does not support this data type. I am returning a double precision"
[1] "floating point, but you must be aware that this could lose precision!"
[1] ">>>> WARNING <<<  attribute GRIB_numberOfPoints is an 8-byte value, but R"
[1] "does not support this data type. I am returning a double precision"
[1] "floating point, but you must be aware that this could lose precision!"
[1] ">>>> WARNING <<<  attribute GRIB_stepUnits is an 8-byte value, but R"
[1] "does not support this data type. I am returning 

In [ ]:
longitude <- ncvar_get(nc_data_BiharRain, "longitude")
latitude <- ncvar_get(nc_data_BiharRain, "latitude")
time <- ncvar_get(nc_data_BiharRain, "valid_time")

# Extract precipitation data (3D array: [lon, lat, time])
tp <- ncvar_get(nc_data_BiharRain, "tp")

# Convert time to POSIXct (UNIX time format)
time_units <- ncatt_get(nc_data_BiharRain, "valid_time", "units")$value
origin_string <- sub("seconds since ", "", time_units)
time <- as.POSIXct(time, origin = origin_string, tz = "UTC")

# Convert time to "yearmon" format using the zoo package
yearmon_time <- as.yearmon(time)

# Create combinations of coordinates using expand.grid
grid1 <- expand.grid(
  longitude = longitude,
  latitude = latitude,
  time = yearmon_time  # Use the yearmon format here
)

# Flatten the 3D tp array (note the correct dimension order!)
grid1$tp <- as.vector(aperm(tp, c(3, 2, 1)))  # Reorder to [time, lat, lon] before flattening

# Preview the data
head(grid1)

,longitude,latitude,time,tp
,<dbl[1d]>,<dbl[1d]>,<yearmon>,<dbl>
1,83.32400,27.287,Jan 1990,5.602837e-06
2,83.57405,27.287,Jan 1990,1.978318e-03
3,83.82411,27.287,Jan 1990,4.875396e-04
4,84.07416,27.287,Jan 1990,7.063659e-04
5,84.32421,27.287,Jan 1990,6.039276e-03
6,84.57426,27.287,Jan 1990,5.667608e-03


In [ ]:
grid1_spatial_avg_tp <- grid1 %>%
  group_by(time) %>%
  summarise(avg_tp_m = mean(tp, na.rm = TRUE),   # Calculate mean temperature for each time
            median_tp_m = median(tp, na.rm = TRUE),
            sd_tp_m = sd(tp, na.rm = TRUE))      # Calculate standard deviation for each time

# Step 2: Add year and month columns
grid1_spatial_avg_tp <- grid1_spatial_avg_tp %>%
  mutate(
    year = year(time),
    month = month(time, label = TRUE, abbr = TRUE),
    yearmonth = as.yearmon(time),  # creates yearmon format like "Jan 1990"
    cv_tp_m = (sd_tp_m / avg_tp_m)  # Calculate CV
  )

# Preview the updated data
grid1_spatial_avg_tp

time,avg_tp_m,median_tp_m,sd_tp_m,year,month,yearmonth,cv_tp_m
<yearmon>,<dbl>,<dbl>,<dbl>,<dbl>,<ord>,<yearmon>,<dbl>
Jan 1990,0.003834972,0.0010662056,0.005067160,1990,Jan,Jan 1990,1.321303
Feb 1990,0.003826709,0.0010823515,0.005092604,1990,Feb,Feb 1990,1.330805
Mar 1990,0.003886567,0.0012042514,0.005166419,1990,Mar,Mar 1990,1.329301
Apr 1990,0.003500327,0.0011505149,0.004464045,1990,Apr,Apr 1990,1.275322
May 1990,0.003351532,0.0009338572,0.004552753,1990,May,May 1990,1.358409
Jun 1990,0.003367026,0.0010444260,0.004385105,1990,Jun,Jun 1990,1.302367
Jul 1990,0.003283415,0.0009636202,0.004385694,1990,Jul,Jul 1990,1.335711
Aug 1990,0.003093348,0.0007762162,0.004234078,1990,Aug,Aug 1990,1.368769
Sep 1990,0.003183087,0.0008151482,0.004315471,1990,Sep,Sep 1990,1.355750


In [ ]:
grid1_spatial_avg_tp1 <- grid1_spatial_avg_tp |> select(yearmonth, year, month, median_tp_m)

In [ ]:
grid1_spatial_avg_tp1

yearmonth,year,month,median_tp_m
<yearmon>,<dbl>,<ord>,<dbl>
Jan 1990,1990,Jan,0.0010662056
Feb 1990,1990,Feb,0.0010823515
Mar 1990,1990,Mar,0.0012042514
Apr 1990,1990,Apr,0.0011505149
May 1990,1990,May,0.0009338572
Jun 1990,1990,Jun,0.0010444260
Jul 1990,1990,Jul,0.0009636202
Aug 1990,1990,Aug,0.0007762162
Sep 1990,1990,Sep,0.0008151482


In [ ]:
Bihar_Temp_Rain <- left_join(df_spatial_avg, grid1_spatial_avg_tp1, by = c("yearmonth", "year", "month"))
Bihar_Temp_Rain

time,avg_temp_c,sd_temp_c,cv_temp_c,avg_rel_humidity,sd_rel_humidity,cv_rel_humidity,avg_spec_humidity,sd_spec_humidity,cv_spec_humidity,year,month,yearmonth,median_tp_m
<yearmon>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<ord>,<yearmon>,<dbl>
Jan 1990,19.33634,0.7966955,0.04120199,64.17878,5.541296,0.08634156,0.008153793,0.0003931764,0.04822006,1990,Jan,Jan 1990,0.0010662056
Feb 1990,20.27928,0.8034896,0.03962120,63.96904,4.067586,0.06358679,0.008622389,0.0005780407,0.06703951,1990,Feb,Feb 1990,0.0010823515
Mar 1990,24.04149,0.8983804,0.03736792,53.87720,6.599863,0.12249827,0.008888774,0.0009458913,0.10641415,1990,Mar,Mar 1990,0.0012042514
Apr 1990,29.79853,2.0819888,0.06986885,45.24342,11.431962,0.25267680,0.009759053,0.0014753555,0.15117815,1990,Apr,Apr 1990,0.0011505149
May 1990,29.63234,1.6729635,0.05645736,70.41598,7.978035,0.11329864,0.016878349,0.0013107042,0.07765594,1990,May,May 1990,0.0009338572
Jun 1990,30.38284,1.4570396,0.04795600,76.09963,8.002538,0.10515870,0.019070562,0.0011897559,0.06238704,1990,Jun,Jun 1990,0.0010444260
Jul 1990,27.99853,0.4612983,0.01647580,88.04368,1.351272,0.01534775,0.019594474,0.0010068715,0.05138548,1990,Jul,Jul 1990,0.0009636202
Aug 1990,28.64218,0.3796141,0.01325368,84.37242,1.798621,0.02131764,0.019544322,0.0010943722,0.05599438,1990,Aug,Aug 1990,0.0007762162
Sep 1990,27.53694,0.4080383,0.01481785,85.91865,1.257775,0.01463914,0.018716469,0.0011422963,0.06103161,1990,Sep,Sep 1990,0.0008151482


In [ ]:
Bihar_Temp_Rain1 <- Bihar_Temp_Rain |> filter(year >= 1990 & year <= 2024)
Bihar_Temp_Rain1

time,avg_temp_c,sd_temp_c,cv_temp_c,avg_rel_humidity,sd_rel_humidity,cv_rel_humidity,avg_spec_humidity,sd_spec_humidity,cv_spec_humidity,year,month,yearmonth,median_tp_m
<yearmon>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<ord>,<yearmon>,<dbl>
Jan 1990,19.33634,0.7966955,0.04120199,64.17878,5.541296,0.08634156,0.008153793,0.0003931764,0.04822006,1990,Jan,Jan 1990,0.0010662056
Feb 1990,20.27928,0.8034896,0.03962120,63.96904,4.067586,0.06358679,0.008622389,0.0005780407,0.06703951,1990,Feb,Feb 1990,0.0010823515
Mar 1990,24.04149,0.8983804,0.03736792,53.87720,6.599863,0.12249827,0.008888774,0.0009458913,0.10641415,1990,Mar,Mar 1990,0.0012042514
Apr 1990,29.79853,2.0819888,0.06986885,45.24342,11.431962,0.25267680,0.009759053,0.0014753555,0.15117815,1990,Apr,Apr 1990,0.0011505149
May 1990,29.63234,1.6729635,0.05645736,70.41598,7.978035,0.11329864,0.016878349,0.0013107042,0.07765594,1990,May,May 1990,0.0009338572
Jun 1990,30.38284,1.4570396,0.04795600,76.09963,8.002538,0.10515870,0.019070562,0.0011897559,0.06238704,1990,Jun,Jun 1990,0.0010444260
Jul 1990,27.99853,0.4612983,0.01647580,88.04368,1.351272,0.01534775,0.019594474,0.0010068715,0.05138548,1990,Jul,Jul 1990,0.0009636202
Aug 1990,28.64218,0.3796141,0.01325368,84.37242,1.798621,0.02131764,0.019544322,0.0010943722,0.05599438,1990,Aug,Aug 1990,0.0007762162
Sep 1990,27.53694,0.4080383,0.01481785,85.91865,1.257775,0.01463914,0.018716469,0.0011422963,0.06103161,1990,Sep,Sep 1990,0.0008151482


In [ ]:
Bihar_Temp_Rain2 <- Bihar_Temp_Rain1 |> select(!(time)) |> select(yearmonth, year, month, everything())

In [ ]:
Bihar_Temp_Rain2

yearmonth,year,month,avg_temp_c,sd_temp_c,cv_temp_c,avg_rel_humidity,sd_rel_humidity,cv_rel_humidity,avg_spec_humidity,sd_spec_humidity,cv_spec_humidity,median_tp_m
<yearmon>,<dbl>,<ord>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Jan 1990,1990,Jan,19.33634,0.7966955,0.04120199,64.17878,5.541296,0.08634156,0.008153793,0.0003931764,0.04822006,0.0010662056
Feb 1990,1990,Feb,20.27928,0.8034896,0.03962120,63.96904,4.067586,0.06358679,0.008622389,0.0005780407,0.06703951,0.0010823515
Mar 1990,1990,Mar,24.04149,0.8983804,0.03736792,53.87720,6.599863,0.12249827,0.008888774,0.0009458913,0.10641415,0.0012042514
Apr 1990,1990,Apr,29.79853,2.0819888,0.06986885,45.24342,11.431962,0.25267680,0.009759053,0.0014753555,0.15117815,0.0011505149
May 1990,1990,May,29.63234,1.6729635,0.05645736,70.41598,7.978035,0.11329864,0.016878349,0.0013107042,0.07765594,0.0009338572
Jun 1990,1990,Jun,30.38284,1.4570396,0.04795600,76.09963,8.002538,0.10515870,0.019070562,0.0011897559,0.06238704,0.0010444260
Jul 1990,1990,Jul,27.99853,0.4612983,0.01647580,88.04368,1.351272,0.01534775,0.019594474,0.0010068715,0.05138548,0.0009636202
Aug 1990,1990,Aug,28.64218,0.3796141,0.01325368,84.37242,1.798621,0.02131764,0.019544322,0.0010943722,0.05599438,0.0007762162
Sep 1990,1990,Sep,27.53694,0.4080383,0.01481785,85.91865,1.257775,0.01463914,0.018716469,0.0011422963,0.06103161,0.0008151482


In [ ]:
Bihar_Temp_Rain3 <- Bihar_Temp_Rain2 |> mutate(state = "Bihar") |> select(state, everything())
Bihar_Temp_Rain3

state,yearmonth,year,month,avg_temp_c,sd_temp_c,cv_temp_c,avg_rel_humidity,sd_rel_humidity,cv_rel_humidity,avg_spec_humidity,sd_spec_humidity,cv_spec_humidity,median_tp_m
<chr>,<yearmon>,<dbl>,<ord>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Bihar,Jan 1990,1990,Jan,19.33634,0.7966955,0.04120199,64.17878,5.541296,0.08634156,0.008153793,0.0003931764,0.04822006,0.0010662056
Bihar,Feb 1990,1990,Feb,20.27928,0.8034896,0.03962120,63.96904,4.067586,0.06358679,0.008622389,0.0005780407,0.06703951,0.0010823515
Bihar,Mar 1990,1990,Mar,24.04149,0.8983804,0.03736792,53.87720,6.599863,0.12249827,0.008888774,0.0009458913,0.10641415,0.0012042514
Bihar,Apr 1990,1990,Apr,29.79853,2.0819888,0.06986885,45.24342,11.431962,0.25267680,0.009759053,0.0014753555,0.15117815,0.0011505149
Bihar,May 1990,1990,May,29.63234,1.6729635,0.05645736,70.41598,7.978035,0.11329864,0.016878349,0.0013107042,0.07765594,0.0009338572
Bihar,Jun 1990,1990,Jun,30.38284,1.4570396,0.04795600,76.09963,8.002538,0.10515870,0.019070562,0.0011897559,0.06238704,0.0010444260
Bihar,Jul 1990,1990,Jul,27.99853,0.4612983,0.01647580,88.04368,1.351272,0.01534775,0.019594474,0.0010068715,0.05138548,0.0009636202
Bihar,Aug 1990,1990,Aug,28.64218,0.3796141,0.01325368,84.37242,1.798621,0.02131764,0.019544322,0.0010943722,0.05599438,0.0007762162
Bihar,Sep 1990,1990,Sep,27.53694,0.4080383,0.01481785,85.91865,1.257775,0.01463914,0.018716469,0.0011422963,0.06103161,0.0008151482


In [ ]:
write.csv(Bihar_Temp_Rain3, "Bihar_Temp_RainMSR.csv", row.names = FALSE)